In [1]:
import numpy as np
from transformers import BertConfig,BertModel,AutoConfig,AutoTokenizer,BertForNextSentencePrediction
import torch
from torch.nn.functional import softmax
from datasets import load_dataset,load_from_disk
import json


In [2]:
from datasets import load_dataset,load_from_disk
import json

In [3]:

#dataset = load_dataset('trivia_qa', 'rc')
#dataset.save_to_disk('.')

In [ ]:
dataset = load_from_disk(r'../data/trivia_qa_rc_tiny')

In [ ]:
train_dataset = dataset['train']

In [ ]:
def function(item):
    print(len(item['search_results']['search_context']))

In [ ]:
#train_dataset.map(function)

# Useful fields: 
Question:
Answer:
Entity_pages -> wiki_context (multiple contexts)
search_results -> search_context (multiple contexts)

# Planning:
set of documents which contains all of wiki context and search results

for each document:
    Split by sentences using NLTK:
    Tokenize each sentence using bert (wordpiece) embeddings
    Store each tokenized sentence in a ordered dict of {sentence_idx -> tokenized sentence}
    run Bert's NSP over the document to generate sequence of split confidence
    Pick a confidence threshold. Threshold needs to be s.t each segment has less than 512 tokens.
    Assign each sentence to a segment based on threshold.
    
    
    


In [ ]:
for ii,item in enumerate(train_dataset):
    #print(json.dumps(item,indent=4))
    print(len(item['search_results']['search_context']), len(item['entity_pages']['wiki_context']))
    if ii == 7:
         print(json.dumps(item,indent=4))
    if ii ==10:
        break
    

In [ ]:
for item in train_dataset:
    print(json.dumps(item,indent=4))
    #print(item['search_result']['search_context'])
    break

In [ ]:
print(torch.cuda.is_available())

In [ ]:
model_name = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
#config = AutoConfig.from_pretrained(model_name)
model = BertForNextSentencePrediction.from_pretrained(model_name)


In [ ]:
seq_A = 'I like cookies !'
seq_B = 'Do you like them ?'
encoded = tokenizer.encode_plus(seq_A, text_pair=seq_B, return_tensors='pt')


In [ ]:
outputs = model(**encoded, labels=torch.LongTensor([1]))
probs = softmax(outputs.logits, dim=1)
prob_next = probs[0][0]